In [ ]:
from __future__ import division
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
import random
import glob
import cv2
from matplotlib import pyplot as plt

In [ ]:
def load_data(filename):
    f = open(filename, 'r')
    csv_r = csv.reader(f, delimiter='\t')
    data = []
    for row in csv_r:
        data.append([float(elem) for elem in row])
    return  np.array(data)

In [ ]:
# Load Data: data is already normalized
X = load_data('X_train.txt')
print(X.shape)
print(len(X))

# load labels
y = load_data(('y_train.txt'))
y = np.ravel(y)
print(y.shape)

In [ ]:
# Select a balanced portion of vessel/background pixels to tune parameters
all_idx = list(range(len(X)))
idx_ves = list(np.ravel(np.where(y == 1)))
#print(idx_ves.shape)

idx_back = list(set(all_idx)-set(idx_ves))
print(len(idx_ves))
print(len(idx_back))


In [ ]:
random.seed(1)
ves_tuning = random.sample(idx_ves, 101404) 
back_tuning = random.sample(idx_back, 101404)
idx_training = ves_tuning + back_tuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(X[idx_training,:], y[idx_training]);

In [ ]:
# save the model to disk
filename = 'rf_model.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
#Load test 
# Load Data
X = load_data('X_test.txt')
print(X.shape)
print(len(X))

y = load_data(('y_test.txt'))
y = np.ravel(y)
print(y.shape)

In [ ]:
#load the model from disk
loaded_model = pickle.load(open( 'rf_model.sav', 'rb'))
result = loaded_model.score(X, y)
print(result)


In [ ]:
predict = loaded_model.predict(X)
print(predict)

In [ ]:
# Create output folder
output = 'output_svm'
os.mkdir(output)

In [ ]:
# Retrieve name of images 
list_name = list()
images_path = '../../filter/filter_test'
for infile in glob.glob(images_path + '/*.png'):
    p, im_name = os.path.split(infile)
    file, ext = os.path.splitext(im_name)
    list_name.append(file)
    
    
j = 0    
for num, i in enumerate(range(0,len(predict), 8281)):
    im =  np.asarray(predict[j:i+8281])
    im = im.reshape((91,91))
    cv2.imwrite(output + '/'+list_name[num] + '.png' , im*255)
    j = i+8281